# ZHCPA Dingtalk

Resources:
- [Dingtalk Dev Docs](https://open.dingtalk.com/document/orgapp/create-an-approval-form-template?spm=ding_open_doc.document.0.0.53244a97oo0wyS)
- [Dingtalk Backend API Explorer](https://open-dev.dingtalk.com/apiExplorer#/?devType=org&api=workflow_1.0%23ListProcessInstanceIds)
- [Dingtalk Frontend API Explorer](https://open-dev.dingtalk.com/apiExplorer#/jsapi?api=device.notification.extendModal)
- [Dingtalk Admin Dashboard](https://oa.dingtalk.com/admin/portal/oa#?lang=zh_CN&nation=HK&code=b885d25ac75b3ad2a203ab9fcb3d0799)

Schema:
![](assets/schema.png)

In [3]:
import datetime
from op.supabase import supabase, execute_mutation, execute_query

# execute_mutation(supabase.table("opuser").insert({
#     "name": "john wick",
#     "email": "johnwick@example.com",
#     "employee_code": 99,
#     "department": "tax",
#     "rank": "manager"
# }))
# execute_mutation(supabase.table("attendance").insert({
#     "id": 1,
#     "opuser_id": 1,
#     "check_in": True,
#     "miss_check": False,
#     "check_time": datetime.datetime.now().isoformat(),
#     "check_source": "machine"
# }))
# execute_mutation(supabase.table("shift").insert({
#     "start_time": datetime.time(8, 0).isoformat(),
#     "end_time": datetime.time(18, 0).isoformat(),
#     "work_duration": 8,
#     "country": "HK"
# }))
# execute_mutation(supabase.table("opuser_shift_assignment").insert({
#     "opuser_id": 1,
#     "shift_id": 1,
#     "job_code": 101,
#     "assign_date": datetime.date.today().isoformat()
# }))
# execute_mutation(supabase.table("overtime").insert({
#     "opuser_id": 1,
#     "record_create_time": datetime.datetime(2023, 7, 31).isoformat(),
#     "status": "APPROVED",
#     "overtime_start_time": datetime.datetime(2023, 7, 31, 18, 0).isoformat(),
#     "overtime_end_time": datetime.datetime(2023, 7, 31, 19, 0).isoformat(),
#     "location": "Megabox",
#     "opuser_remark": "working hard"
# }))
# execute_mutation(supabase.table("leave").insert({  # pyright: ignore
#     "opuser_id": 1,
#     "record_create_time": datetime.datetime(2023, 7, 31).isoformat(),
#     "status": "REJECTED",
#     "leave_type": "annual leave",
#     "leave_start_time": datetime.datetime(2023, 7, 31, 8, 0).isoformat(),
#     "leave_end_time": datetime.datetime(2023, 7, 31, 12, 0).isoformat(),
#     "leave_duration": 4,
#     "leave_duration_unit": "hour",
#     "opuser_remark": "party hard"
# }))

In [4]:
import pandas as pd

In [6]:
opuser_pd = pd.DataFrame(execute_query(supabase.table("opuser").select("*")))    # pyright: ignore
opuser_pd

""


In [7]:
attendance_pd = pd.DataFrame(execute_query(supabase.table("attendance").select("*")))    # pyright: ignore
attendance_pd

""


In [8]:
shift_pd = pd.DataFrame(execute_query(supabase.table("shift").select("*")))    # pyright: ignore
shift_pd

,id,start_time,end_time,work_duration,holiday_scheme_id
0,1,08:00:00+00,18:00:00+00,8,1
1,2,10:00:00+08,18:00:00+08,7,1


In [9]:
opuser_shift_assignment_pd = pd.DataFrame(execute_query(supabase.table("opuser_shift_assignment").select("*")))    # pyright: ignore
opuser_shift_assignment_pd

,id,opuser_id,shift_id,job_code,assign_date
0,2,None,1,101,2023-07-01
1,3,None,2,1000,2023-08-10


In [10]:
overtime_pd = pd.DataFrame(execute_query(supabase.table("overtime").select("*")))    # pyright: ignore
overtime_pd

,id,opuser_id,record_create_time,status,overtime_start_time,overtime_end_time,location,opuser_remark
0,1,None,2023-07-31T00:00:00,APPROVED,2023-07-31T18:00:00,2023-07-31T19:00:00,Megabox,working hard


In [11]:
leave_pd = pd.DataFrame(execute_query(supabase.table("leave").select("*")))    # pyright: ignore
leave_pd

,id,opuser_id,record_create_time,status,leave_type,leave_start_time,leave_end_time,leave_duration,leave_duration_unit,opuser_remark
0,1,None,2023-07-31T00:00:00,REJECTED,annual leave,2023-07-31T08:00:00,2023-07-31T12:00:00,4,hour,party hard


In [19]:
execute_query(supabase.table("attendance").select("*"))

[{'id': 2,
  'opuser_id': 1,
  'check_in': False,
  'miss_check': False,
  'check_time': '2023-08-19T18:03:00',
  'check_source': 'machine'}]

In [24]:
execute_query(supabase.table("holiday").select("*"))

[]

In [30]:
execute_query(supabase.table("holiday_scheme").select("*"))

[{'id': 1, 'name': 'hong kong'}]

In [31]:
execute_mutation(supabase.table("holiday_scheme").delete().eq("id", "1"))

Exception: Request not ok {'data': {'code': '23502', 'details': 'Failing row contains (1, 08:00:00+00, 18:00:00+00, 8, null).', 'hint': None, 'message': 'null value in column "holiday_scheme_id" of relation "shift" violates not-null constraint'}, 'status_code': 400}

In [5]:
execute_mutation(supabase.table("opuser").delete().eq("id", "2"))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
from op.supabase import execute_delete_mutation

execute_delete_mutation(supabase.table("holiday").delete().eq("id", "1"))

In [12]:
from pprint import pprint


pprint(supabase.table("opuser_daily_attendance_view").select("*").execute()["data"])

{'code': '21000',
 'details': None,
 'hint': None,
 'message': 'more than one row returned by a subquery used as an expression'}
